# 获取数据


In [66]:
from mxnet import gluon
from mxnet import nd
from mxnet import autograd
def transform(data,label):
    return data.astype('float32')/255,label.astype('float32')
minist_train=gluon.data.vision.FashionMNIST(root='fashion-mnist/',train=True,transform=transform)
minist_test=gluon.data.vision.FashionMNIST(root='fashion-mnist/',train=False,transform=transform)
batch_size=256 
train_data=gluon.data.DataLoader(minist_train,batch_size=batch_size,shuffle=True)
test_data=gluon.data.DataLoader(minist_test,batch_size=batch_size,shuffle=True)

In [67]:
from mxnet import nd 
num_inputs,num_hidden ,num_outputs=28*28,256,10 
weight_scale=.01 
'''
loc:float
　　概率分布的均值，对应着整个分布的中心center
scale:float
　　概率分布的标准差，对应于分布的宽度，scale越大越矮胖，scale越小，越瘦高
'''
w1 ,b1 = nd.random_normal(shape=(num_inputs,num_hidden),scale=weight_scale),nd.zeros(num_hidden)
w2 ,b2 = nd.random_normal(shape=(num_hidden,num_outputs),scale=weight_scale),nd.zeros(num_outputs)
params = [w1,b1,w2,b2]
for param in params:
    param.attach_grad()


# 在两层隐含层间定义relu激活函数，使模型可拟合非线性函数

In [68]:
def relu(X):
    return nd.maximum(X,0)

# 定义模型，全连接+Relu

In [69]:
def net(X):
    X=X.reshape((-1,num_inputs))
    h1=relu(nd.dot(X,w1)+b1)
    output=nd.dot(h1,w2)+b2 
    return output

# Softmax+交叉熵损失函数

In [70]:
from mxnet import gluon
softmax_cross_entropy = gluon.loss.SoftmaxCrossEntropyLoss()

# 定义精确度和优化函数

In [71]:
def accuracy(output,label):
    return nd.mean(output.argmax(axis=1)==label).asscalar()
def evaluate_accuracy(test_data,net):
    loss=0.
    for data,label in test_data:
        output=net(data) 
        loss+=accuracy(output,label)
    return loss/len(test_data)
def SGD(params,lr):
    for param in params:
        param[:]=param-lr*param.grad

# 训练

In [76]:
from mxnet import autograd
epochs=4
learning_rate=.1
for e in range(1,epochs+1):
    train_loss,train_acc=0.,0. 
    for data,label in train_data:
        with autograd.record():
            output=net(data)
            loss=softmax_cross_entropy(output,label)
        loss.backward()
        SGD(params,learning_rate/batch_size)
        train_loss+=nd.mean(loss).asscalar()
        train_acc+=accuracy(output,label)
    test_acc=evaluate_accuracy(test_data,net)
    print('epoch %d,loss:%f..,train_accuracy:%f,test_accuracy:%f..'%(e,train_loss/len(train_data),train_acc/len(train_data),
                                                                    test_acc/len(test_data)))

epoch 1,loss:0.358499..,train_accuracy:0.871941,test_accuracy:0.021719..
epoch 2,loss:0.350339..,train_accuracy:0.873986,test_accuracy:0.021743..
epoch 3,loss:0.343806..,train_accuracy:0.875770,test_accuracy:0.021934..
epoch 4,loss:0.338015..,train_accuracy:0.877438,test_accuracy:0.021936..


In [75]:
[p.shape for p in params]

[(784, 256), (256,), (256, 10), (10,)]

In [77]:
import d2lzh as d2l
help(d2l)

Help on package d2lzh:

NAME
    d2lzh

PACKAGE CONTENTS
    utils

DATA
    VOC_CLASSES = ['background', 'aeroplane', 'bicycle', 'bird', 'boat', '...
    VOC_COLORMAP = [[0, 0, 0], [128, 0, 0], [0, 128, 0], [128, 128, 0], [0...

VERSION
    0.8.11

FILE
    d:\software\python3.7\lib\site-packages\d2lzh\__init__.py


